In [15]:
import geopandas as gpd

# full file to compare other files to
fullgdf = gpd.read_parquet('../../data/wijkenbuurten_2023_clean.parquet')

# individual files
wijken = gpd.read_file('../../data/cbs_shp/Wijken/wijken_2023_V1.shp')
buurten = gpd.read_file('../../data/cbs_shp/Buurten/buurten_2023_V1.shp')
gemeenten = gpd.read_file('../../data/cbs_shp/Gemeenten/gemeenten_2023_V1.shp')

In [16]:
# Replace column names to match other data
replacements = {
    "BU_CODE": "buurtcode",
    "BU_NAAM": "buurtnaam",
    "WK_CODE": "wijkcode",
    "WK_NAAM": "wijknaam",
    "GM_CODE": "gemeentecode",
    "GM_NAAM": "gemeentenaam",
}

wijken.columns = [replacements.get(col, col) for col in wijken.columns]
buurten.columns = [replacements.get(col, col) for col in buurten.columns]
gemeenten.columns = [replacements.get(col, col) for col in gemeenten.columns]

In [17]:
# Set correct crs
wijken = wijken.to_crs(fullgdf.crs)
buurten = buurten.to_crs(fullgdf.crs)
gemeenten = gemeenten.to_crs(fullgdf.crs)

In [18]:
# remove rows that are not in the full file
count_w, count_b, count_g = len(wijken), len(buurten), len(gemeenten)

wijken = wijken[wijken.wijkcode.isin(fullgdf.wijkcode)]
buurten = buurten[buurten.buurtcode.isin(fullgdf.buurtcode)]
gemeenten = gemeenten[gemeenten.gemeentecode.isin(fullgdf.gemeentecode)]

# remove rows where H2O is 'ja'
wijken = wijken[wijken.H2O != 'JA']
buurten = buurten[buurten.H2O != 'JA']
gemeenten = gemeenten[gemeenten.H2O != 'JA']

print(f"Removed {count_w - len(wijken)} rows from wijken")
print(f"Removed {count_b - len(buurten)} rows from buurten")
print(f"Removed {count_g - len(gemeenten)} rows from gemeenten")

Removed 110 rows from wijken
Removed 572 rows from buurten
Removed 82 rows from gemeenten


In [19]:
# reset indices
wijken.reset_index(drop=True, inplace=True)
buurten.reset_index(drop=True, inplace=True)
gemeenten.reset_index(drop=True, inplace=True)

In [20]:
# save files
wijken.to_parquet('../../data/wijken.parquet')
buurten.to_parquet('../../data/buurten.parquet')
gemeenten.to_parquet('../../data/gemeenten.parquet')